TEST 

In [ ]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import torch 
import random 

#pytoch imports

import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


In [ ]:
b1 =[torch.LongTensor([i for i in range(5*j,5*j+5-random.randint(0,2))]) for j in range(1,4)]
#b2 =[torch.LongTensor([i for i in range(5*j,5*j+5)]) for j in range(1,4)]

b1_lenghts = [len(ex) for ex in b1]
    

b1

b1_lenghts

In [ ]:
max_tokens = 7
b1[0] = nn.ConstantPad1d((0,max_tokens-b1[0].shape[0]),0)(b1[0])

b1

In [ ]:
padded_b1 = rnn.pad_sequence(b1,batch_first=True,padding_value=0)

padded_b1
padded_b1.shape

In [ ]:
fake_matrix = [[random.randint(1,7) for _ in range(4)] for _ in range(19)]
fake_matrix.insert(0,[0,0,0,0])
fake_matrix 

In [ ]:
fake_matrix = torch.Tensor(fake_matrix)   #the embedding matrix 
_ , embedding_dim = fake_matrix.shape 
emb_layer = nn.Embedding.from_pretrained(fake_matrix, freeze=False, padding_idx = 0)

In [ ]:
emb = emb_layer(padded_b1)

emb.shape
emb

In [ ]:
packed_embeds = pack_padded_sequence(emb,b1_lenghts,batch_first=True, enforce_sorted=False)

packed_embeds

In [ ]:
re_padded_embeds = pad_packed_sequence(packed_embeds,batch_first=True)

re_padded_embeds

In [ ]:
emb_reshaped = emb.permute(0,2,1)

emb_reshaped.shape

emb_reshaped

In [ ]:
dense_layer = nn.Linear(max_tokens,1)
sentence_emb = dense_layer(emb_reshaped)

sentence_emb

sentence_emb = sentence_emb.squeeze(2)

sentence_emb
sentence_emb.shape


In [ ]:
class model(nn.Module):
    """
        Class defining our model architecture  
    """
    def __init__(self, emb_matrix , pad_idx: int, max_tokens: int) :
        super().__init__()

        self.embedding_layer, self.word_embedding_dim = self.create_emb_layer(emb_matrix,pad_idx)

        self.pad_idx = pad_idx

        self.max_tokens = max_tokens

        self.rnn = nn.LSTM(self.word_embedding_dim, self.word_embedding_dim,bidirectional = False,batch_first = True) 

    def create_emb_layer(self,weights_matrix, pad_idx : int):
        
        matrix = torch.Tensor(weights_matrix)   #the embedding matrix 
        _ , embedding_dim = matrix.shape 
        emb_layer = nn.Embedding.from_pretrained(matrix, freeze=True, padding_idx = pad_idx)   #load pretrained weights in the layer and make it non-trainable 
        
        return emb_layer, embedding_dim


    def forward(self, claims, claim_lengths):

        claims = claims.copy()
        
        claims[0] = nn.ConstantPad1d((0,max_tokens-claims[0].shape[0]),0)(claims[0])

        padded_claims = rnn.pad_sequence(claims,batch_first = True, padding_value = self.pad_idx)         

        embed_claims = self.embedding_layer(padded_claims)

        packed_embeds = pack_padded_sequence(embed_claims, claim_lengths, batch_first=True, enforce_sorted=False)   #pack the sentences batch so that no unnecessary computation is performed for padding tokens
        
        packed_out, (hn,_)  = self.rnn(packed_embeds)

        unpacked_out, l = pad_packed_sequence(packed_out,batch_first=True)

        return unpacked_out,l 
    
        

In [ ]:
a =[torch.LongTensor([i for i in range(5*j,5*j+5-random.randint(0,2))]) for j in range(1,4)]
a_lenths = [len(ex) for ex in a]

In [ ]:
a

In [ ]:
m = model(fake_matrix,0,max_tokens)

p,l = m(a,a_lenths)

p.shape

p


a1 = p.sum(dim=1).div(p.count_nonzero(dim=1))
a2 = torch.sum(p,dim=1).div(1.7)

a1.shape

a2.shape

In [ ]:
a1

a2

In [ ]:
m1 = torch.cat((a1,a2),dim=1)
m1

In [ ]:
m2 = torch.stack((a1,a2), dim=0).sum(dim=0)
m2

In [ ]:
m3 = torch.stack((a1,a2), dim=0).mean(dim=0)
m3

In [ ]:
b1 = torch.Tensor([0.7736, -0.0804,  0.0381,  0.0499])

b2 = torch.Tensor([0.1595,  0.0741,  0.0667,  0.1454])

In [ ]:
c = torch.cat((b1,b2))

torch.cat((c,b1))

In [ ]:
import torch.nn.functional as F

cosine_sim = F.cosine_similarity(a1,a2)

cosine_sim.shape

cosine_sim = cosine_sim.unsqueeze(-1)

cosine_sim.shape

f = torch.cat((m1,cosine_sim),dim=1)

f.shape

f



In [ ]:
classifier = nn.Linear(9,1)

labels = classifier(f)

labels

labels = labels.squeeze()

labels 
labels.shape

In [ ]:

true = torch.Tensor([1,0,1])

loss = nn.BCEWithLogitsLoss()

l = loss(labels.squeeze(),true)




In [ ]:
prova = [0,1,0]

prova = torch.Tensor(prova)

prova.long()

In [ ]:
classes = (labels>0 ).float()

classes

In [ ]:
def ciao(n :int):

    def a():
        print(n)

    def b():
        print(n+1)
    
    if n%2 == 0:
        a()
    else :
        b()
    

ciao(5)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

#compute accuracy and f1-score 
def acc_and_f1(y_pred: torch.LongTensor, y_true: torch.LongTensor):
    """
        Compute accuracy and f1-score for an epoch 
    """
    # correct = y_pred.eq(y_true)          
    # acc = correct.sum()/y_true.shape[0] 

    acc = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true,y_pred,average='macro')

    return acc,f1

#construct y_true and y_pred lists to be passed to acc_and_f1 function, but based on majority voting strategy
def majority_voting(y_pred,y_true,y_ids):
    """
        Input: the list of predicted labels, the list of corresponing true labels, the list of claim ids to compute majority voting 
        Output : the list of predicted labels via majority voting, the list of true labels (one for each claim id)
    """
    for id in y_ids:
        mask = y_ids[y_ids == id ]
        

    return 

def acc_and_f1_majority(y_pred,y_true,y_ids):

    y_pred,y_true = majority_voting(y_pred,y_true,y_ids)

    acc, f1 = acc_and_f1(y_pred, y_true)

    return acc, f1

In [ ]:
import numpy as np

In [ ]:
pred = torch.LongTensor([0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0])
true = torch.LongTensor([1,0,1,0,0,1,1,1,1,0,1,0,0,1,1,1,1,0,1,0,0,1,1,1])
ids  = torch.LongTensor([3,2,3,1,2,7,3,7,3,2,3,1,2,7,3,7,3,2,3,1,2,7,3,7])

y_true = []
y_pred = []
ids.unique()

for id in ids.unique():
    mask = torch.isin(ids,id)
    y_true.append(true[mask][0].item())
    y_pred.append(torch.argmax(torch.bincount(pred[mask])).item())

y_true
y_pred